In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Parkinsons_Disease"
cohort = "GSE71220"

# Input paths
in_trait_dir = "../../input/GEO/Parkinsons_Disease"
in_cohort_dir = "../../input/GEO/Parkinsons_Disease/GSE71220"

# Output paths
out_data_file = "../../output/preprocess/Parkinsons_Disease/GSE71220.csv"
out_gene_data_file = "../../output/preprocess/Parkinsons_Disease/gene_data/GSE71220.csv"
out_clinical_data_file = "../../output/preprocess/Parkinsons_Disease/clinical_data/GSE71220.csv"
json_path = "../../output/preprocess/Parkinsons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The effect of statins on blood gene expression in COPD"
!Series_summary	"Background: COPD is currently the fourth leading cause of death worldwide and predicted to rank third by 2020. Statins are commonly used lipid lowering agents with documented benefits on cardiovascular morbidity and mortality, and have also been shown to have pleiotropic effects including anti-inflammatory and anti-oxidant activity.   Objective: Identify a gene signature associated with statin use in the blood of COPD patients, and identify molecular mechanisms and pathways underpinning this signature that could explain any potential benefits in COPD.  Methods: Whole blood gene expression was measured on 168 statin users and 452 non-users from the ECLIPSE (Evaluation of COPD Longitudinally to Identify Predictive Surrogate Endpoints) study. Gene expression was measured using the Affymetrix  Human Gene 1.1 ST microarray chips. Factor Analysis for Robust Microarray Summarization

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this appears to be gene expression data
# The information mentions "Whole blood gene expression was measured on 168 statin users and 452 non-users"
# and "Gene expression was measured using the Affymetrix Human Gene 1.1 ST microarray chips"
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For Parkinson's Disease trait:
# From the background information, this dataset is about COPD (Chronic Obstructive Pulmonary Disease),
# not Parkinson's Disease. The dataset doesn't contain information relevant to Parkinson's Disease.
trait_row = None  # No relevant trait data for Parkinson's Disease

# For age:
# Age is available in row 2
age_row = 2

# For gender:
# Gender is available in row 3 labeled as "Sex"
gender_row = 3

# 2.2 Data Type Conversion

def convert_trait(value):
    """Convert disease status to binary, where 1 represents having Parkinson's Disease and 0 represents control."""
    # This dataset doesn't have PD information, so this function won't be used
    # But we define it for completeness
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # This dataset doesn't contain PD data
    return None

def convert_age(value):
    """Convert age to continuous numeric value."""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary, where 0 represents female and 1 represents male."""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Since trait_row is None, we don't have relevant trait data available for Parkinson's Disease
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is None (no relevant PD data), we skip this step
if trait_row is not None:
    # Get clinical dataframe by selecting relevant features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the results
    preview = preview_df(clinical_df)
    print("Preview of clinical data:")
    print(preview)
    
    # Save clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['7892501', '7892504', '7892507', '7892508', '7892509', '7892510',
       '7892514', '7892515', '7892516', '7892517', '7892520', '7892521',
       '7892527', '7892530', '7892531', '7892533', '7892534', '7892535',
       '7892536', '7892538'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers appear to be numeric probe IDs from a microarray platform
# These are not human gene symbols, but rather platform-specific identifiers
# They need to be mapped to gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['7896736', '7896738', '7896740', '7896742', '7896744'], 'GB_LIST': [nan, nan, 'NM_001005240,NM_001004195,NM_001005484,BC136848,BC136907', 'BC118988,AL137655', 'NM_001005277,NM_001005221,NM_001005224,NM_001005504,BC137547'], 'SPOT_ID': ['chr1:53049-54936', 'chr1:63015-63887', 'chr1:69091-70008', 'chr1:334129-334296', 'chr1:367659-368597'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'RANGE_GB': ['NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10', 'NC_000001.10'], 'RANGE_STRAND': ['+', '+', '+', '+', '+'], 'RANGE_START': [53049.0, 63015.0, 69091.0, 334129.0, 367659.0], 'RANGE_STOP': [54936.0, 63887.0, 70008.0, 334296.0, 368597.0], 'total_probes': [7.0, 31.0, 24.0, 6.0, 36.0], 'gene_assignment': ['---', '---', 'NM_001005240 // OR4F17 // olfactory receptor, family 4, subfamily F, member 17 // 19p13.3 // 81099 /// NM_001004195 // OR4F4 // olfactory receptor, family 4, subfamily F, member 4 // 15q26.3 // 26682 /// NM_001005484 // OR4F5 //

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in the gene annotation DataFrame contain identifiers and gene symbols
# From the preview, we can see:
# - 'ID' column contains numeric identifiers that match the gene expression data's row indices
# - 'gene_assignment' column contains gene symbols in a complex format that needs parsing

# 2. Get a gene mapping dataframe
# Create a mapping from probe IDs to gene symbols
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='gene_assignment')

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression
gene_data = apply_gene_mapping(gene_data, mapping_data)

# Print the first few gene symbols after mapping to verify successful conversion
print("First 20 gene symbols after mapping:")
print(gene_data.index[:20])

# Get the dimensions of the gene expression data
print(f"Gene expression data shape: {gene_data.shape}")


First 20 gene symbols after mapping:
Index(['A-', 'A-52', 'A-I', 'A1', 'A1-', 'A10', 'A11', 'A12', 'A13', 'A14',
       'A18', 'A19', 'A2', 'A20', 'A22', 'A23', 'A24', 'A2M', 'A3', 'A33'],
      dtype='object', name='Gene')
Gene expression data shape: (23114, 617)


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# Since we determined in Step 2 that trait_row is None (this dataset doesn't contain Parkinson's Disease data),
# we cannot link clinical and genetic data for our target trait
print("This dataset doesn't contain relevant data for Parkinson's Disease.")

# Set is_trait_available to False since we don't have PD trait data
is_trait_available = False

# Validate using is_final=False since we're doing an initial validation for a dataset without our target trait
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available
)

print("Dataset is not usable for target trait analysis. Linked data will not be saved.")

Gene data shape after normalization: (7901, 617)


Normalized gene data saved to ../../output/preprocess/Parkinsons_Disease/gene_data/GSE71220.csv
This dataset doesn't contain relevant data for Parkinson's Disease.
Dataset is not usable for target trait analysis. Linked data will not be saved.
